In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math
from nltk import pos_tag, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
data = pd.read_csv("Our_dataset/winemag-data-130k-v2.csv")
data.drop(columns=["Unnamed: 0"],inplace=True)

[nltk_data] Downloading package punkt to /home/simone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/simone/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/simone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/simone/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# DATASET AND DESCRIPTION PRE PROCESSING 

In [7]:
data = data.drop_duplicates(["description","price","points","taster_name","title"])
len(data)

119988

In [8]:
#check some stuff
"""
missingOnlyName = 0
missingOnlyTwitter = 0
missingBoth = 0
lis = []
for idx,row in data.iterrows():
    #print(row["taster_name"])
    if(pd.isnull(row["taster_name"]) and (not pd.isnull(row["taster_twitter_handle"]))):
        missingOnlyName = missingOnlyName + 1
    if(pd.isnull(row["taster_twitter_handle"]) and (not pd.isnull(row["taster_name"]))):
        missingOnlyTwitter = missingOnlyTwitter + 1
    if(pd.isnull(row["taster_name"]) and pd.isnull(row["taster_twitter_handle"])):
        missingBoth = missingBoth + 1
    couple = [row["taster_name"],row["taster_twitter_handle"]]
    if not couple in lis:
        lis.append(couple)
print(missingOnlyName,missingOnlyTwitter,missingBoth)
print(lis,len(lis))
"""
data.drop(["taster_twitter_handle"],axis=1,inplace=True)

In [9]:
avgPriceWinery = data.groupby("winery")#.price.mean()
#avgPriceWinery.get_group("1+1=3").price.mean()
data['price'] = data.apply(
    lambda row: avgPriceWinery.get_group(row["winery"]).price.mean() if pd.isnull(row['price']) else row['price'],
    axis=1
)
data["price"].fillna(data["price"].mean(),inplace=True)

#prende solo recensioni con autore conosciuto, ha senso come cosa?
#data.dropna(subset=["taster_name"],inplace=True)

data.fillna("unknown",inplace=True)

In [10]:
actualData = data.groupby("country").filter(lambda x: len(x)>600) 
len(actualData)

117652

In [11]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()

/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41


In [12]:
taggedDescriptions = []
for idx, row in actualData.iterrows():
    desc = row["description"]
    tokens = word_tokenize(desc)
    tagged = pos_tag(tokens)
    taggedDescriptions.append(tagged)

In [13]:

descriptions = []
for item in taggedDescriptions:
    processed = ""
    for el in item:
        if el[1].startswith("N") or el[1].startswith("J"):  #se iniziano con N o J sono nomi o aggettivi, quello che ci serve di più
            processed = processed + " " + el[0]
    descriptions.append(processed)

In [14]:
#descriptions[3]

In [15]:
actualData["description"] = descriptions

/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
#lower case everything
actualData['description'] = actualData['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))

/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
actualData

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,aromas tropical fruit broom brimstone herb pal...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,ripe fruity wine firm tannins juicy red berry ...,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy flavors lime flesh dominate green ...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom aroma...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41
5,Spain,blackberry typical navarran whiff green herbs ...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,42
6,Italy,bright informal aromas candied berry white pep...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,31
7,France,dry restrained wine offers profusion acidity f...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,21
8,Germany,savory thyme notes accent sunnier flavors pres...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,24
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,30


In [19]:
#se vuoi fermarti qua, senza stemming 
actualData.to_csv("dataAfterPOS.csv", encoding='utf-8', index=False)

# DESCRIPTION STEMMING 

In [20]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
stemmedDataset = actualData

In [21]:
stemDescriptions = []
for idx, row in stemmedDataset.iterrows():
    desc = ""
    for el in row["description"].split():
        desc = desc + stemmer.stem(el) + " "
    stemDescriptions.append(desc)

In [22]:
stemDescriptions[0]

'aroma tropic fruit broom brimston herb palat express unripen appl citru dri sage brisk acid '

In [23]:
stemmedDataset["description"] = stemDescriptions
stemmedDataset["description"]

/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0         aroma tropic fruit broom brimston herb palat e...
1         ripe fruiti wine firm tannin juici red berri f...
2         tart snappi flavor lime flesh domin green pine...
3         pineappl rind lemon pith orang blossom aroma p...
4         much regular bottl rough tannic rustic earthi ...
5         blackberri typic navarran whiff green herb cas...
6         bright inform aroma candi berri white pepper s...
7         dri restrain wine offer profus acid firm textu...
8         savori thyme note accent sunnier flavor preser...
9         great depth flavor fresh appl pear fruit touch...
10        soft suppl plum oaki structur cabernet % merlo...
11        dri wine spici tight taut textur miner charact...
12        wine chalki tannic backbon otherwis juici expl...
13        oak oak-driven aroma coffe bean espresso cocon...
14        year gener tradit wineri leaner style classic ...
15        zesti orang peel appl note mineral-ton riesl o...
16        bake plum molass balsam vinega

In [24]:
stemmedDataset.drop(["word_count"],axis=1,inplace=True)
#final dataset to be used as BASE
stemmedDataset.to_csv("stemmed.csv", encoding='utf-8', index=False)

/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [25]:
stemmedDataset

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,aroma tropic fruit broom brimston herb palat e...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,ripe fruiti wine firm tannin juici red berri f...,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,tart snappi flavor lime flesh domin green pine...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,pineappl rind lemon pith orang blossom aroma p...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,bright inform aroma candi berri white pepper s...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,France,dri restrain wine offer profus acid firm textu...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,Germany,savori thyme note accent sunnier flavor preser...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,France,great depth flavor fresh appl pear fruit touch...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


# LEMMATISATION (useless, don't do ) 

In [ ]:
#non porta differenze, chissene
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

#example text text = 'What can I say about this place. The staff of these restaurants is nice and the eggplant is not bad'

class Splitter(object):
    """
    split the document into sentences and tokenize each sentence
    """
    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):
        """
        out : ['What', 'can', 'I', 'say', 'about', 'this', 'place', '.']
        """
        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(sent) for sent in sentences]
        return tokens


class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens

lemmatizer = WordNetLemmatizer()
splitter = Splitter()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

#step 1 split document into sentence followed by tokenization
tokens = splitter.split(data.iloc[100]["description"])

#step 2 lemmatization using pos tagger 
lemma_pos_token = lemmatization_using_pos_tagger.pos_tag(tokens)
print(lemma_pos_token)


lemmatization non cambia un cazzo, lasciamo stemming 